# T1.1 - Solution
## Authors:
- Leonardo Kaplan 1212509
- Nino Fabrizio Tiriticco Lizardo 1113203

In [ ]:
# Pacotes usados
import pandas as pd # Para pegar os dados dos arquivos
from IPython.display import display # Para mostrar mais de uma informação em uma mesma célula
import ast # Para transformar string/object em estruturas de dados (listas, dicionários, ...)
import numpy as np # Para obter o total de valores por um atributo, se um data frame está vazio
import matplotlib.pyplot as plt # Para plotar gráficos
import matplotlib # Para plotar gráficos
from pylab import * # Para criar pie chart

In [ ]:
# Carregando dados de cada um dos arquivos
DataQualisRaw = pd.read_excel('in/Qualis CC 2013-2016.xlsx')
DataDocentesRaw = pd.read_csv('in/docentes.csv')
DataDiscentesRaw = pd.read_csv('in/discentes.csv')
DataProducaoRaw = pd.read_csv('in/producao.csv')
DataTrabalhosRaw = pd.read_csv('in/trabalhos.csv')

# Tirando duplicatas
DataQualisRaw = DataQualisRaw.drop_duplicates(subset=DataQualisRaw.columns.values, keep=False)
DataQualisRaw = DataQualisRaw.reset_index(drop = True)
DataDocentesRaw = DataDocentesRaw.drop_duplicates(subset=DataDocentesRaw.columns.values, keep=False)
DataDocentesRaw = DataDocentesRaw.reset_index(drop = True)
DataDiscentesRaw = DataDiscentesRaw.drop_duplicates(subset=DataDiscentesRaw.columns.values, keep=False)
DataDiscentesRaw = DataDiscentesRaw.reset_index(drop = True)
DataProducaoRaw = DataProducaoRaw.drop_duplicates(subset=DataProducaoRaw.columns.values, keep=False)
DataProducaoRaw = DataProducaoRaw.reset_index(drop = True)
DataTrabalhosRaw = DataTrabalhosRaw.drop_duplicates(subset=DataTrabalhosRaw.columns.values, keep=False)
DataTrabalhosRaw = DataTrabalhosRaw.reset_index(drop = True)

## Análise prévia dos dados:

In [ ]:
# Pela função info, parece que temos todos os dados disponíveis para cada coluna (sem valores nulos)
display(DataQualisRaw.info())

# Pelas 5 primeiras tuplas, os dados parecem não precisar tratamento
DataQualisRaw.head()

In [ ]:
# Pela função info, parece que temos todos os dados disponíveis para cada coluna (sem valores nulos)
display(DataDocentesRaw.info())

# Pelas 5 primeiras tuplas, os dados parecem não precisar tratamento
DataDocentesRaw.head()

In [ ]:
# Pela função info, parece que temos dados faltando para algumas das colunas (há valores nulos)
display(DataDiscentesRaw.info())

# Pelas tabela, devemos precisar tratar os dados da coluna "abrev". Importante ressaltar que alguns dos dados em "orientadores" parecem representar listas de dicionários através de string/object.
DataDiscentesRaw.head()

In [ ]:
# A função info nos mostra que temos valores nulos em algumas das colunas (total de valores não-nulos menor que o total de tuplas da tabela)
display(DataProducaoRaw.info())

pd.set_option('display.max_columns', 35) # Para poder visualizar todas as colunas deste data frame

# As 5 primeiras tuplas da tabela nos mostram que pelo menos alguns dados precisam ser tratados nas colunas:
# - dict_paper_autores (os dados dessa coluna na verdade representam listas de dicionários) [VER PRÓXIMA CÉLULA]
# - paper_autores (os dados dessa coluna na verdade representam listas de strings/objects) [VER CÉLULA PRÓXIMA À SEGUINTE]
# - doi
# - periodico (talvez?)
# - ano (número fracionário para ano faz sentido? cuidado, faz sim se formos considerar como período [ex.: 2018.1 e 2018.2])
DataProducaoRaw.head()

In [ ]:
# Vemos que uma célula da segunda coluna de DataProducaoRaw é uma string/object que representa uma lista de dicionários
display(DataProducaoRaw['dict_paper_autores'][0])

# Transformando uma dessas string/objects em uma lista de dicionários
dictionaryList = ast.literal_eval(DataProducaoRaw['dict_paper_autores'][0])

display(dictionaryList) # Lista de dicionários
display(dictionaryList[0]) # Dicionário
display(dictionaryList[0]['categoria']) # Valor atribuído à chave 'categoria'
dictionaryList[0]['nome'] # Valor atribuído à chave 'nome'

In [ ]:
# Vemos que uma célula da coluna 'paper_autores' de DataProducaoRaw é uma string/object que representa uma lista de strings/objects
display(DataProducaoRaw['paper_autores'][0])

stringList = ast.literal_eval(DataProducaoRaw['paper_autores'][0])

display(stringList) # Lista de strings/objects
display(stringList[0]) # Uma (TADAM!) string/object

In [ ]:
# Pela função info, parece que temos valores nulos em algumas das colunas
display(DataTrabalhosRaw.info())

# Pelas 5 primeiras tuplas, parece que precisaremos tratar os dados das colunas:
# - keywords
# - palavras-chave
# - paginas (número fracionário para páginas faz sentido?)
# - programa (talvez?)
DataTrabalhosRaw.head()

# Perguntas:
## 1) Quantos professores (docentes) havia em cada instituição em 2017, em cada quadro (permanente, colaborador)?

In [ ]:
# DataDocentesRaw contém os dados necessários para responder a pergunta (colunas "ies" e "categoria")
# Verificando quais as instituições, parecem OK
display(DataDocentesRaw['ies'].unique())

# Verificando quais as categorias, parecem OK
DataDocentesRaw['categoria'].unique()

In [ ]:
# Montando nosso data frame
docentesByIESDF = pd.DataFrame(columns=('IES', 'Permanentes', 'Colaboradores'))
docentesByIESDF['IES'] = DataDocentesRaw['ies'].unique()
docentesByIESDF['Permanentes'] = 0
docentesByIESDF['Colaboradores'] = 0

# Percorrendo o data frame original para fazer a contagem dos tipos de docente por instituição
for indx in range(0, len(DataDocentesRaw)):
    if DataDocentesRaw['categoria'][indx] == 'PERMANENTE':
        docentesByIESDF.loc[docentesByIESDF['IES'] == DataDocentesRaw['ies'][indx], 'Permanentes'] += 1
    elif DataDocentesRaw['categoria'][indx] == 'COLABORADOR':
        docentesByIESDF.loc[docentesByIESDF['IES'] == DataDocentesRaw['ies'][indx], 'Colaboradores'] += 1
    else: # Caso encontremos um imprevisto
        print("\'categoria\' error! Value:", DataDocentesRaw['categoria'][indx], " Row:", indx)
        
# O resultado obtido
docentesByIESDF

In [ ]:
# Montando um gráfico de barras horizontais empilhadas do resultado obtido
x_labels = docentesByIESDF['IES']
x = range(len(x_labels))
y = docentesByIESDF['Colaboradores']
x2 = []
for item in x:
    x2.append(item)
y2 = docentesByIESDF['Permanentes']

bar_width = 0.35

plt.figure()
plt.barh(x2, y2, height=bar_width, label="Permanente", color='#4466cc')
plt.barh(x, y, height=bar_width, label ="Colaborador", color='#cc6644')

plt.title('Professores x Programa')
plt.yticks(x, x_labels)
plt.legend(loc=4, frameon=True, title='Tipo de professor')

plt.show()

## 2) Quantos alunos (discentes) de Mestrado/Doutorado havia em cada programa em 2017?

In [ ]:
# DataDiscentesRaw contém os dados necessários para responder a pergunta (colunas "nivel" e "programa")
# Pegando apenas as colunas que queremos
discentesDFQ2 = DataDiscentesRaw[['nivel','programa']]

display(discentesDFQ2.info())
discentesDFQ2.head()

In [ ]:
# Verificando valores em "nivel", vemos que existe pelo menos um valor 'Graduação' que não nos interessa
display(discentesDFQ2['nivel'].unique())

# Verificando valores em "programa"
display(discentesDFQ2['programa'].unique())

# Eliminando valores 'Graduação', obtemos uma redução considerável de tuplas
discentesDFQ2 = discentesDFQ2.loc[discentesDFQ2['nivel'] != 'Graduação']
discentesDFQ2 = discentesDFQ2.reset_index(drop = True)
discentesDFQ2.info()

In [ ]:
# Montando nosso data frame
discentesByPrograma = pd.DataFrame(columns=('Programa', 'Mestrandos', 'Doutorandos'))
discentesByPrograma['Programa'] = discentesDFQ2['programa'].unique()
discentesByPrograma['Mestrandos'] = 0
discentesByPrograma['Doutorandos'] = 0

# Percorrendo o data frame limpo para fazer a contagem dos tipos de nível por programa
for indx in range(0, len(discentesDFQ2)):
    if discentesDFQ2['nivel'][indx] == 'Mestrado':
        discentesByPrograma.loc[discentesByPrograma['Programa'] == discentesDFQ2['programa'][indx], 'Mestrandos'] += 1
    elif discentesDFQ2['nivel'][indx] == 'Doutorado':
        discentesByPrograma.loc[discentesByPrograma['Programa'] == discentesDFQ2['programa'][indx], 'Doutorandos'] += 1
    else: # Caso encontremos um imprevisto
        print("\'nivel\' error! Value:", discentesDFQ2['nivel'][indx], " Row:", indx)
        
# O resultado obtido
discentesByPrograma

In [ ]:
# Montando um gráfico de barras horizontais do resultado obtido
x_labels = discentesByPrograma['Programa']
x = range(len(x_labels))
y = discentesByPrograma['Mestrandos']

bar_width = 0.35

x2 = []
for item in x:
    x2.append(item + bar_width)
y2 = discentesByPrograma['Doutorandos']

plt.figure()
plt.barh(x, y, height=bar_width, label ="Mestrado", color='#cc6644')
plt.barh(x2, y2, height=bar_width, label="Doutorado", color='#4466cc')

plt.title('Alunos x Programa')
plt.yticks(x, x_labels)
plt.legend(loc=4, frameon=True, title='Nível de aluno')

plt.show()

## 3) Qual foi a taxa de alunos de Mestrado/Doutorado por professor do quadro permanente em cada programa em 2017?

In [ ]:
# Vamos usar os data frames DataDocentesRaw e DataDiscentesRaw
# Começando por DataDiscentesRaw
discentesDFQ3 = DataDiscentesRaw[['nivel','programa', 'orientador']]
display(discentesDFQ3.info()) # Vemos que coluna 'orientador' tem muitos valores faltando

# Retirando todas as colunas onde 'orientador' está nulo
discentesDFQ3 = discentesDFQ3.loc[discentesDFQ3['orientador'].isnull() == False]
discentesDFQ3 = discentesDFQ3.reset_index(drop = True)
display(discentesDFQ3.info()) # Agora todas as colunas possuem a mesma quantidade de linhas, originalmente sendo a de 'orientador' (o menor valor que tínhamos antes)

# Vemos que acabamos tirando o valor 'Graduação' em 'nivel' no meio dessa limpeza
display(discentesDFQ3['nivel'].unique())

discentesDFQ3.head()

In [ ]:
# Agora vamos com DataDocentesRaw
docentesDFQ3 = DataDocentesRaw[['categoria','nome']]
display(docentesDFQ3.info()) # Parece tudo OK

# Eliminando valores 'COLABORADOR'
docentesDFQ3 = docentesDFQ3.loc[docentesDFQ3['categoria'] != 'COLABORADOR']
docentesDFQ3 = docentesDFQ3.reset_index(drop = True)
display(docentesDFQ3.info()) # A redução não foi tão expressiva

docentesDFQ3.head()

In [ ]:
# Montando o data frame de resultado
taxasByPrograma = pd.DataFrame(columns=('Programa', 'Taxa Mestrandos/Professores permanentes', 'Taxa Doutorandos/Professores permanentes'))
taxasByPrograma['Programa'] = discentesDFQ3['programa'].unique()
taxasByPrograma['Taxa Mestrandos/Professores permanentes'] = 0
taxasByPrograma['Taxa Doutorandos/Professores permanentes'] = 0

# Procurando valores por valor 'programa'
for programa in taxasByPrograma['Programa']:
    # Reduzindo meu data frame 'discentes' para o valor 'programa' da iteração
    tempDiscentesDFQ3 = discentesDFQ3.loc[discentesDFQ3['programa'] == programa]
    
    # Reduzindo o data frame temporário ainda mais para valores 'orientador' com valor 'PERMANENTE'
    tempDiscentesDFQ3 = tempDiscentesDFQ3.loc[tempDiscentesDFQ3['orientador'].isin(docentesDFQ3['nome'])]
    tempDiscentesDFQ3 = tempDiscentesDFQ3.reset_index(drop = True)
    
    # Listas para guardar nomes do respectivo nivel do orientador
    orientadoresMestrado = []
    orientadoresDoutorado = []
    
    # Percorrendo o data frame temporário para fazer a contagem da quantidade de discentes por nível, guardando nome do docente
    for indx in range(0, len(tempDiscentesDFQ3)):
        if tempDiscentesDFQ3['nivel'][indx] == 'Mestrado':
            taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Mestrandos/Professores permanentes'] += 1
            if not orientadoresMestrado or tempDiscentesDFQ3['orientador'][indx] not in orientadoresMestrado:
                orientadoresMestrado.append(tempDiscentesDFQ3['orientador'][indx])
        elif tempDiscentesDFQ3['nivel'][indx] == 'Doutorado':
            taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Doutorandos/Professores permanentes'] += 1
            if not orientadoresDoutorado or tempDiscentesDFQ3['orientador'][indx] not in orientadoresDoutorado:
                orientadoresDoutorado.append(tempDiscentesDFQ3['orientador'][indx])
    
    # Calculando taxa "tipo discentes/docentes"
    if orientadoresMestrado:
        taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Mestrandos/Professores permanentes'] = taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Doutorandos/Professores permanentes'] / len(orientadoresMestrado)
    if orientadoresDoutorado:
        taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Doutorandos/Professores permanentes'] = taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Doutorandos/Professores permanentes'] / len(orientadoresDoutorado)

# O resultado obtido
taxasByPrograma

In [ ]:
# Montando resultados em gráfico de barras verticais, um gráfico para cada nível por programa
matplotlib.rcParams.update({'font.size': 12})

fig, ax = plt.subplots(figsize=(16,8))
y_pos = [i for i in range(0,len(taxasByPrograma['Programa']))]
x_values = list(taxasByPrograma['Taxa Mestrandos/Professores permanentes'])
x2_values = list(taxasByPrograma['Taxa Doutorandos/Professores permanentes'])
bar_width = 0.35
y2_pos = [i + bar_width for i in range(0,len(taxasByPrograma['Programa']))]

ax.set_title("Taxa \'alunos/professores permanentes\' x Programa")
ax.barh(y_pos, x_values, height=bar_width, label='Mestrado', color='#cc6644')
ax.barh(y2_pos, x2_values, height=bar_width, label='Doutorado', color='#4466cc')
ax.set_yticks(y_pos)
ax.set_yticklabels(list(taxasByPrograma['Programa']))
plt.xlim((0,max(x_values)*1.2))
rects = ax.patches

# For each bar: Place a label
for rect in rects:
    # Get X and Y placement of label from rect.
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2

    # Use Y value as label and format number with one decimal place
    label = "{:}".format(x_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(5, 5),              #  Shift label (horizontally,vertically)
        textcoords="offset points", # Interpret `xytext` as offset in points
        ha='left',                  # Horizontal label alignment
        va='top')                   # Vertical label alignment

plt.legend(loc=4, frameon=True, title='Nível calculado')    

plt.show()

## 4) Qual foi a distribuição de alunos de Mestrado/Doutorado pelos professores de cada programa em 2017?

In [ ]:
# Vamos voltar a usar as tabelas 'docente' e 'discente'
# Fazendo a limpeza e extraindo o que nos interessa
discentesDFQ4 = DataDiscentesRaw.loc[DataDiscentesRaw['orientador'].isnull() == False][['nivel','programa', 'orientador']]
discentesDFQ4 = discentesDFQ4.reset_index(drop = True)

display(discentesDFQ4.info())
display(discentesDFQ4.head())

docentesDFQ4 = DataDocentesRaw[['nome']]

display(docentesDFQ4.info())
docentesDFQ4.head()

In [ ]:
# Montando o dicionário resultado (key == programa ; value == data frame de distribuição)
distribuicoesByPrograma = {}

# Procurando por valor 'programa'
for programa in discentesDFQ4['programa'].unique():
    distMestradoDF = pd.DataFrame(columns=('Professor', 'Qtd mestrandos'))
    distDoutoradoDF = pd.DataFrame(columns=('Professor', 'Qtd doutorandos'))
    
    # Reduzindo meu data frame 'discentes' para o valor 'programa' da iteração
    tempDiscentesDFQ4 = discentesDFQ4.loc[discentesDFQ4['programa'] == programa]
    
    # Reduzindo o data frame temporário ainda mais para valores 'orientador' com valor 'PERMANENTE'
    tempDiscentesDFQ4 = tempDiscentesDFQ4.loc[tempDiscentesDFQ4['orientador'].isin(docentesDFQ4['nome'])]
    tempDiscentesDFQ4 = tempDiscentesDFQ4.reset_index(drop = True)
    
    # Dicionários para guardar nomes de orientadores e qtd de alunos (key == orientador ; value == qtd alunos)
    orientadoresMestrado = {}
    orientadoresDoutorado = {}
    
    # Percorrendo o data frame temporário para guardar os valores no respectivo dicionário
    for indx in range(0, len(tempDiscentesDFQ4)):
        if tempDiscentesDFQ4['nivel'][indx] == 'Mestrado':
            if not orientadoresMestrado or tempDiscentesDFQ4['orientador'][indx] not in orientadoresMestrado:
                orientadoresMestrado.update({tempDiscentesDFQ4['orientador'][indx]: 1})
            elif orientadoresMestrado and tempDiscentesDFQ4['orientador'][indx] in orientadoresMestrado:
                orientadoresMestrado[tempDiscentesDFQ4['orientador'][indx]] += 1
        elif tempDiscentesDFQ4['nivel'][indx] == 'Doutorado':
            if not orientadoresDoutorado or tempDiscentesDFQ4['orientador'][indx] not in orientadoresDoutorado:
                orientadoresDoutorado.update({tempDiscentesDFQ4['orientador'][indx]: 1})
            elif orientadoresDoutorado and tempDiscentesDFQ4['orientador'][indx] in orientadoresDoutorado:
                orientadoresDoutorado[tempDiscentesDFQ4['orientador'][indx]] += 1
    
    # Guardando os valores calculados dos dicionários, pelo orientador
    for orientador, qtdAlunos in orientadoresMestrado.items():
        distMestradoDF.loc[len(distMestradoDF)] = [orientador, qtdAlunos]
    for orientador, qtdAlunos in orientadoresDoutorado.items():
        distDoutoradoDF.loc[len(distDoutoradoDF)] = [orientador, qtdAlunos]
    
    # Juntando valores da coluna pelo nome do orientador
    distMestradoDF = distMestradoDF.groupby('Professor').first().reset_index()
    distMestradoDF = distMestradoDF.groupby('Qtd mestrandos').size().reset_index(name='Qtd professores')
    distDoutoradoDF = distDoutoradoDF.groupby('Professor').first().reset_index()
    distDoutoradoDF = distDoutoradoDF.groupby('Qtd doutorandos').size().reset_index(name='Qtd professores')
    
    # Reordenando as colunas
    columns = distMestradoDF.columns.tolist()
    columns = columns[-1:] + columns[:-1]
    distMestradoDF = distMestradoDF[columns]
    columns = distDoutoradoDF.columns.tolist()
    columns = columns[-1:] + columns[:-1]
    distDoutoradoDF = distDoutoradoDF[columns]
    
    # Reordenando os valores em ordem decrescente
    distMestradoDF = distMestradoDF.sort_values(['Qtd professores', 'Qtd mestrandos'], ascending=[True, True])
    distMestradoDF = distMestradoDF.reset_index(drop = True)
    distMestradoDF = distMestradoDF.fillna(0)
    distDoutoradoDF = distDoutoradoDF.sort_values(['Qtd professores', 'Qtd doutorandos'], ascending=[True, True])
    distDoutoradoDF = distDoutoradoDF.reset_index(drop = True)
    distDoutoradoDF = distDoutoradoDF.fillna(0)
    
    # Guardando o data frame final no dicionário de resultado
    distribuicoesByPrograma.update({programa: [distMestradoDF, distDoutoradoDF]})

In [ ]:
# Mostrando o que achamos
for programa, distribuicoes in distribuicoesByPrograma.items():
    print(programa, ':')
    display(distribuicoes[0])
    display(distribuicoes[1])
    print()

In [ ]:
# Montando resultados em gráficos de barras horizontais com os respectivo valores no final de cada barra
matplotlib.rcParams.update({'font.size': 10})

for programa, distribuicoes in distribuicoesByPrograma.items():
    x_labels = distribuicoes[0]['Qtd professores']
    x = range(len(x_labels))
    y = distribuicoes[0]['Qtd mestrandos']

    plt.figure(figsize=(5, 5))
    plt.title(programa)
    plt.bar(x, y, align='center', width = 0.45, color='#cc6644')
    plt.xticks(x, x_labels)
    plt.yticks(y, distribuicoes[0]['Qtd mestrandos'])
    plt.xlabel('Qtd Professores')
    plt.ylabel('Qtd Mestrandos')

    plt.show()
    
    x_labels = distribuicoes[1]['Qtd professores']
    x = range(len(x_labels))
    y = distribuicoes[1]['Qtd doutorandos']

    plt.figure(figsize=(5, 5))
    plt.title(programa)
    plt.bar(x, y, align='center', width = 0.45, color='#4466cc')
    plt.xticks(x, x_labels)
    plt.yticks(y, distribuicoes[1]['Qtd doutorandos'])
    plt.xlabel('Qtd Professores')
    plt.ylabel('Qtd Doutorandos')

    plt.show()

## 5) Quantos alunos de Mestrado/Doutorado defenderam suas dissertações/teses em 2017 (arquivo trabalhos.csv)?

In [ ]:
# Vamos usar DataTrabalhosRaw
# Vemos que faltam dados no data frame
DataTrabalhosRaw.info()

In [ ]:
# Pegando as colunas que nos interessam e eliminando valores nulos através de uma das colunas do respectivo data frame
trabalhosDFQ5 = DataTrabalhosRaw.loc[DataTrabalhosRaw['autor'].isnull() == False][['autor', 'tipo', 'data_defesa']]
trabalhosDFQ5 = trabalhosDFQ5.sort_values(['autor', 'tipo'], ascending=[True, True])
trabalhosDFQ5 = trabalhosDFQ5.reset_index(drop = True)

display(trabalhosDFQ5.head())

# Vemos que felizmente nos desfizemos de todos os valores nulos. A coluna "data_defesa" foi deixada para nos certificarmos que
# todos os discentes aqui presentes realizaram as defesas
trabalhosDFQ5.info()

In [ ]:
# Em trabalhosDFQ5 temos apenas Tese e Dissertação, como queremos
display(trabalhosDFQ5['tipo'].unique())

In [ ]:
# Calculando o total de Mestrandos e Doutorandos
quantityMestrandos = len(trabalhosDFQ5.loc[trabalhosDFQ5['tipo'] == 'DISSERTAÇÃO'])
quantityDoutorandos = len(trabalhosDFQ5.loc[trabalhosDFQ5['tipo'] == 'TESE'])

print("Total Mestrandos:", quantityMestrandos)
print("Total Doutorandos:", quantityDoutorandos)

In [ ]:
# Montando um gráfico de barras verticais dos resultados obtidos
x_labels = trabalhosDFQ5['tipo'].unique()
x = range(len(x_labels))
y = [quantityMestrandos, quantityDoutorandos]

plt.figure(figsize=(5, 5))
plt.title('Quantidade de alunos que realizaram defesa')
plt.bar(x, y, align='center', width = 0.15, color=['#cc6644', '#4466cc'])
plt.xticks(x, x_labels)

plt.show()

## 6) Como os trabalhos de Mestrado/Doutorado defendidos em 2017 foram distribuídos pelas áreas de pesquisa dos programas?

In [ ]:
# Para responder a pergunta, usaremos de novo DataTrabalhosRaw
# Limpando e pegando os dados que nos interessam
trabalhosDFQ6 = DataTrabalhosRaw.loc[DataTrabalhosRaw['autor'].isnull() == False][['autor', 'programa', 'area', 'tipo', 'data_defesa']]
trabalhosDFQ6 = trabalhosDFQ6.reset_index(drop = True)

display(trabalhosDFQ6.info())
trabalhosDFQ6.head()

In [ ]:
# Montando nosso data frame
tipoByArea = pd.DataFrame(columns=('Área', 'Mestrandos', 'Doutorandos'))
tipoByArea['Área'] = trabalhosDFQ6['area'].unique()
tipoByArea['Mestrandos'] = 0
tipoByArea['Doutorandos'] = 0

# Fazendo a contagem em relação ao tipo pra cada área
for indx in range(0, len(trabalhosDFQ6)):
    if trabalhosDFQ6['tipo'][indx] == 'DISSERTAÇÃO':
        tipoByArea.loc[tipoByArea['Área'] == trabalhosDFQ6['area'][indx], 'Mestrandos'] += 1
    elif trabalhosDFQ6['tipo'][indx] == 'TESE':
        tipoByArea.loc[tipoByArea['Área'] == trabalhosDFQ6['area'][indx], 'Doutorandos'] += 1

# Nosso data frame resultante
display(tipoByArea)

In [ ]:
# Vemos que a linha 9 contém "dado vazio" para área, retiremos
display(tipoByArea.loc[(tipoByArea['Área'] == '-')])

tipoByArea = tipoByArea.drop(tipoByArea.index[[9]])
tipoByArea = tipoByArea.reset_index(drop = True)

In [ ]:
tipoByArea = tipoByArea.sort_values('Área', ascending=True)
tipoByArea = tipoByArea.reset_index(drop = True)
tipoByArea

In [ ]:
# Montando resultado em gráfico de barras horizontais com o respectivo valor no final de cada barra
matplotlib.rcParams.update({'font.size': 8})

fig, ax = plt.subplots(figsize=(16,8))
y_pos = [i for i in range(0,len(tipoByArea['Área']))]
x_values = list(tipoByArea['Mestrandos'])
x2_values = list(tipoByArea['Doutorandos'])
bar_width = 0.35
y2_pos = [i + bar_width for i in range(0,len(tipoByArea['Área']))]

ax.set_title("Qtd trabalhos x Área")
ax.barh(y_pos, x_values, height=bar_width, label='Mestrado', color='#cc6644')
ax.barh(y2_pos, x2_values, height=bar_width, label='Doutorado', color='#4466cc')
ax.set_yticks(y_pos)
ax.set_yticklabels(list(tipoByArea['Área']))
plt.xlim((0,max(x_values)*1.2))
rects = ax.patches

# For each bar: Place a label
for rect in rects:
    # Get X and Y placement of label from rect.
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2

    # Use Y value as label and format number with one decimal place
    label = "{:}".format(x_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(5, 5),              #  Shift label (horizontally,vertically)
        textcoords="offset points", # Interpret `xytext` as offset in points
        ha='left',                  # Horizontal label alignment
        va='top')                   # Vertical label alignment

plt.legend(loc=4, frameon=True, title='Tipo de trabalho')    

plt.show()

## 7) Como as defesas de Mestrado/Doutorado foram distribuídas ao longo do ano de 2017?

In [ ]:
# Para responder a pergunta, usaremos de novo DataTrabalhosRaw
# Pegando os atributos que nos interessa, fazendo as devidas limpezas/conversões
trabalhosDFQ7 = DataTrabalhosRaw.loc[(DataTrabalhosRaw['data_defesa'].isnull() == False)][['autor', 'tipo', 'data_defesa']]
trabalhosDFQ7['data_defesa'] = pd.to_datetime(trabalhosDFQ7['data_defesa'])
trabalhosDFQ7 = trabalhosDFQ7.sort_values(['data_defesa', 'tipo'], ascending=[True, True])
trabalhosDFQ7 = trabalhosDFQ7.reset_index(drop = True)

display(trabalhosDFQ7.info())
trabalhosDFQ7.head()

In [ ]:
# Montando nosso data frame
tipoByMes = pd.DataFrame(columns=('Mes', 'Mestrandos', 'Doutorandos'))
tipoByMes['Mes'] = trabalhosDFQ7['data_defesa'].dt.month.unique()
tipoByMes['Mestrandos'] = 0
tipoByMes['Doutorandos'] = 0

# Fazendo a contagem em relação ao tipo pra cada mês
for indx in range(0, len(trabalhosDFQ7)):
    if trabalhosDFQ7['tipo'][indx] == 'DISSERTAÇÃO':
        tipoByMes.loc[tipoByMes['Mes'] == trabalhosDFQ7['data_defesa'].dt.month[indx], 'Mestrandos'] += 1
    elif trabalhosDFQ7['tipo'][indx] == 'TESE':
        tipoByMes.loc[tipoByMes['Mes'] == trabalhosDFQ7['data_defesa'].dt.month[indx], 'Doutorandos'] += 1

# Nosso data frame resultante
tipoByMes

In [ ]:
# Montando gráfico de linhas com os resultados
plt.figure()

x_label = tipoByMes['Mes']
y_values = tipoByMes['Mestrandos']
y2_values = tipoByMes['Doutorandos']
plt.plot(x_label, y_values, '-o',  label='Mestrado')
plt.plot(x_label, y2_values, '-o', label='Doutorado')
    
# Ajustando a margem inferior do gráfico
plt.subplots_adjust(bottom=0.25)

ax = plt.gca()
ax.set_xlabel('Mês')
ax.set_ylabel('Quantidade')
ax.set_title('Defesas x Mês em 2017')
plt.legend(loc=1, frameon=True, title='Tipo de nível')
mes_xticks = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
plt.xticks(x_label, mes_xticks, rotation=45)
ax.stem(x_label, y_values, 'k', markerfmt=' ', linefmt='--')
ax.stem(x_label, y2_values, 'k', markerfmt=' ', linefmt='--')

plt.show()

## 8) Qual fração de alunos de Mestrado/Doutorado (do total em cada programa) defendeu em 2017?

In [ ]:
# Para responder a pergunta, usaremos tabelas 'trabalhos' e o data frame resultado da questão 2
# Pegando os atributos que nos interessa, fazendo as devidas limpezas
trabalhosDFQ8 = DataTrabalhosRaw.loc[(DataTrabalhosRaw['data_defesa'].isnull() == False)][['programa', 'tipo', 'data_defesa']]
trabalhosDFQ8 = trabalhosDFQ8.reset_index(drop = True)

display(trabalhosDFQ8.info())
trabalhosDFQ8.head()

In [ ]:
# Montando o data frame resultado
defesasByPrograma = pd.DataFrame(columns=('Programa', 'Total defesa Mestrandos', 'Total defesa Doutorandos'))
defesasByPrograma['Programa'] = trabalhosDFQ8['programa'].unique()
defesasByPrograma['Total defesa Mestrandos'] = 0
defesasByPrograma['Total defesa Doutorandos'] = 0

# Fazendo a contagem do total de defesas por programa
for indx in range(0, len(trabalhosDFQ8)):
    if trabalhosDFQ8['tipo'][indx] == 'DISSERTAÇÃO':
        defesasByPrograma.loc[defesasByPrograma['Programa'] == trabalhosDFQ8['programa'][indx], 'Total defesa Mestrandos'] += 1
    elif trabalhosDFQ8['tipo'][indx] == 'TESE':
        defesasByPrograma.loc[defesasByPrograma['Programa'] == trabalhosDFQ8['programa'][indx], 'Total defesa Doutorandos'] += 1

# Calculando as porcentagens
defesasByPrograma['% Defesa Mestrandos'] = (defesasByPrograma['Total defesa Mestrandos'] / discentesByPrograma['Mestrandos']) * 100
defesasByPrograma['% Defesa Doutorandos'] = (defesasByPrograma['Total defesa Doutorandos'] / discentesByPrograma['Doutorandos']) * 100

# Nossos resultados
defesasByPrograma

In [ ]:
# Montando gráficos pie chart para cada resultado
explode=(0, 0.05)
for indx in range(0, len(defesasByPrograma)):
    labels = 'Defenderam mestrado', 'Não defenderam mestrado'

    # Para mestrado
    figure(1, figsize=(6,6))
    fracs = [defesasByPrograma['% Defesa Mestrandos'][indx], 100-defesasByPrograma['% Defesa Mestrandos'][indx]]
    pie(fracs, explode=explode, labels=labels, autopct='%1.4f%%', shadow=True, startangle=90)
    title(defesasByPrograma['Programa'][indx], bbox={'facecolor':'0.8', 'pad':5})

    show()
    
    labels = 'Defenderam doutorado', 'Não defenderam doutorado'

    # Para doutorado
    figure(1, figsize=(6,6))
    fracs = [defesasByPrograma['% Defesa Doutorandos'][indx], 100-defesasByPrograma['% Defesa Doutorandos'][indx]]
    pie(fracs, explode=explode, labels=labels, autopct='%1.4f%%', shadow=True, startangle=90)
    title(defesasByPrograma['Programa'][indx], bbox={'facecolor':'0.8', 'pad':5})

    show()

## 9) Quantos artigos de periódico/trabalhos em anais foram publicados por cada programa em 2017?

In [ ]:
# Usaremos tabela 'produção' para responder a pergunta
# Pegando apenas as colunas que nos interessam, temos todos os programas, mas há artigos e trabalhos como nulo
producaoDFQ9 = DataProducaoRaw[['programa', 'paper_id', 'periodico', 'anais_titulo']]
display(producaoDFQ9.info())
producaoDFQ9.head()

In [ ]:
# Montando nosso data frame
artigosTrabalhosByPrograma = pd.DataFrame(columns=('Programa', 'Artigos de periódico', 'Trabalhos em anais'))
artigosTrabalhosByPrograma['Programa'] = producaoDFQ9['programa'].unique()
artigosTrabalhosByPrograma['Artigos de periódico'] = 0
artigosTrabalhosByPrograma['Trabalhos em anais'] = 0

# Fazendo a contagem em relação a artigos e trabalhos para o respectivo programa
for indx in range(0, len(producaoDFQ9)):
    if producaoDFQ9['periodico'][indx] is not np.nan:
        artigosTrabalhosByPrograma.loc[artigosTrabalhosByPrograma['Programa'] == producaoDFQ9['programa'][indx], 'Artigos de periódico'] += 1
    if producaoDFQ9['anais_titulo'][indx] is not np.nan:
        artigosTrabalhosByPrograma.loc[artigosTrabalhosByPrograma['Programa'] == producaoDFQ9['programa'][indx], 'Trabalhos em anais'] += 1

# Nosso data frame resultante
artigosTrabalhosByPrograma

In [ ]:
# Montando um gráfico de barras horizontais do resultado obtido
x_labels = artigosTrabalhosByPrograma['Programa']
x = range(len(x_labels))
y = artigosTrabalhosByPrograma['Artigos de periódico']

bar_width = 0.35

x2 = []
for item in x:
    x2.append(item + bar_width)
y2 = artigosTrabalhosByPrograma['Trabalhos em anais']

plt.figure()
plt.barh(x, y, height=bar_width, label ="Periódico", color='#cc6644')
plt.barh(x2, y2, height=bar_width, label="Anais", color='#4466cc')

plt.title('Produtos x Programa')
plt.yticks(x, x_labels)
plt.legend(loc=4, frameon=True, title='Lugares de publicação')

plt.show()

## 10) Qual é a taxa de artigos de periódico/trabalhos em anais por número de docentes permanentes de cada programa em 2017?

In [ ]:
# Usaremos tabela 'produção' para responder a pergunta
# Pegando apenas as colunas que nos interessam, temos todos os programas, mas há artigos e trabalhos como nulo
producaoDFQ10 = DataProducaoRaw[['programa', 'paper_id', 'periodico', 'anais_titulo', 'ies']]
display(producaoDFQ10.info())
display(producaoDFQ10.head())

docentesDFQ10 = DataDocentesRaw.loc[DataDocentesRaw['categoria'] == 'PERMANENTE'][['nome', 'ies']]
display(docentesDFQ10.info())
docentesDFQ10.head()

In [ ]:
# Montando o data frame de resultado
taxasByPrograma = pd.DataFrame(columns=('Programa', 'Taxa artPer/Professores permanentes', 'Taxa trabAnais/Professores permanentes'))
taxasByPrograma['Programa'] = producaoDFQ10['programa'].unique()
taxasByPrograma['Taxa artPer/Professores permanentes'] = 0
taxasByPrograma['Taxa trabAnais/Professores permanentes'] = 0

# Procurando valores por valor 'programa'
for indx in range(0, len(producaoDFQ10)):
    if producaoDFQ10['periodico'][indx] is not np.nan:
        taxasByPrograma.loc[taxasByPrograma['Programa'] == producaoDFQ10['programa'][indx], 'Taxa artPer/Professores permanentes'] += 1
    if producaoDFQ10['anais_titulo'][indx] is not np.nan:
        taxasByPrograma.loc[taxasByPrograma['Programa'] == producaoDFQ10['programa'][indx], 'Taxa trabAnais/Professores permanentes'] += 1

# Calculando taxas "qtd artigos/docentes" e "qtd trabalhos/docentes"
for programa in taxasByPrograma['Programa']:
    ies = producaoDFQ10.loc[producaoDFQ10['programa'] == programa, 'ies'].unique()
    
    # Reduzindo meu data frame 'docentes' para o valor 'ies' da iteração
    tempDocentesDF = docentesDFQ10.loc[docentesDFQ10['ies'] == ies[0]]
    
    # Reduzindo o data frame temporário ainda mais para valores 'orientador' com valor 'PERMANENTE'
    tempDocentesDF = tempDocentesDF.loc[tempDocentesDF['nome'].isin(docentesDFQ10['nome'])]
    tempDocentesDF = tempDocentesDF.reset_index(drop = True)
    
    taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa artPer/Professores permanentes'] = taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa artPer/Professores permanentes'] / len(tempDocentesDF)
    taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa trabAnais/Professores permanentes'] = taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa trabAnais/Professores permanentes'] / len(tempDocentesDF)

# O resultado obtido
taxasByPrograma

In [ ]:
# Montando resultados em gráfico de barras verticais, um gráfico para cada nível por programa
matplotlib.rcParams.update({'font.size': 12})

fig, ax = plt.subplots(figsize=(16,8))
y_pos = [i for i in range(0,len(taxasByPrograma['Programa']))]
x_values = list(taxasByPrograma['Taxa artPer/Professores permanentes'])
x2_values = list(taxasByPrograma['Taxa trabAnais/Professores permanentes'])
bar_width = 0.35
y2_pos = [i + bar_width for i in range(0,len(taxasByPrograma['Programa']))]

ax.set_title("Taxa \'produções/professores permanentes\' x Programa")
ax.barh(y_pos, x_values, height=bar_width, label='Artigo de periódico', color='#cc6644')
ax.barh(y2_pos, x2_values, height=bar_width, label='Trabalho em anal', color='#4466cc')
ax.set_yticks(y_pos)
ax.set_yticklabels(list(taxasByPrograma['Programa']))
plt.xlim((0,7))
rects = ax.patches

# For each bar: Place a label
for rect in rects:
    # Get X and Y placement of label from rect.
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2

    # Use Y value as label and format number with one decimal place
    label = "{:}".format(x_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(5, 5),              #  Shift label (horizontally,vertically)
        textcoords="offset points", # Interpret `xytext` as offset in points
        ha='left',                  # Horizontal label alignment
        va='top')                   # Vertical label alignment

plt.legend(loc=1, frameon=True, title='Tipo de produção')    

plt.show()

## 11) Qual fração de artigos de periódico/trabalhos em anais publicados em 2017 teve a coautoria de discentes?

In [ ]:
# Usaremos tabela 'produção' para responder a pergunta
# Pegando as colunas que nos interessam
producaoDFQ11 = DataProducaoRaw[['num_docentes', 'num_discentes', 'num_externos', 'paper_id', 'periodico', 'anais_titulo']]
display(producaoDFQ11.info())
producaoDFQ11.head()

In [ ]:
# Existe um valor sem docente e externo. Como queremos coautoria com discentes, precisamos de pelo menos um participante para
# pelo menos um deles. Logo, retiremos essa tupla.
display(producaoDFQ11.loc[(producaoDFQ11['num_docentes'] == 0) & (producaoDFQ11['num_externos'] == 0)])

producaoDFQ11 = producaoDFQ11.drop(producaoDFQ11.index[[2556]])
producaoDFQ11 = producaoDFQ11.reset_index(drop = True)

In [ ]:
# Declarando e inicializando as variáveis que usaremos
artigosDiscentePercent = 0
trabalhosDiscentePercent = 0
artigosTotal = 0
trabalhosTotal = 0

# Fazendo a contagem do total de artigos e trabalhos e de coautoria de discente para cada um deles
for indx in range(0, len(producaoDFQ11)):
    if producaoDFQ11['periodico'][indx] is not np.nan:
        artigosTotal += 1
        if producaoDFQ11['num_discentes'][indx] != 0:
            artigosDiscentePercent += 1
    if producaoDFQ11['anais_titulo'][indx] is not np.nan:
        trabalhosTotal += 1
        if producaoDFQ11['num_discentes'][indx] != 0:
            trabalhosDiscentePercent += 1

# Calculando as porcentagens
artigosDiscentePercent = (artigosDiscentePercent / artigosTotal) * 100
trabalhosDiscentePercent = (trabalhosDiscentePercent / trabalhosTotal) * 100

print("\nArtigos de periódico com coautoria de discente: ", artigosDiscentePercent, "%\nTrabalhos em anais com coautoria de discente: ", trabalhosDiscentePercent, "%")

In [ ]:
# Montando gráficos pie chart para cada resultado
labels = 'Com discente', 'Sem discente'
explode=(0, 0.05)

# Artigos de periódico com coautoria de discente
figure(1, figsize=(6,6))
fracs = [artigosDiscentePercent, 100-artigosDiscentePercent]
pie(fracs, explode=explode, labels=labels, autopct='%1.4f%%', shadow=True, startangle=90)
title('Taxa de coautoria de discentes em artigos de periódico', bbox={'facecolor':'0.8', 'pad':5})

show()

# Trabalhos em anais com coautoria de discente
figure(1, figsize=(6,6))
fracs2 = [trabalhosDiscentePercent, 100-trabalhosDiscentePercent]
pie(fracs2, explode=explode, labels=labels, autopct='%1.4f%%', shadow=True, startangle=90)
title('Taxas de coautoria discentes em trabalhos em anais', bbox={'facecolor':'0.8', 'pad':5})

show()

## 12) Qual fração de artigos de periódico/trabalhos em anais publicados em 2017 teve a coautoria de participantes externos?

In [ ]:
# Usaremos tabela 'produção' para responder a pergunta
# Pegando as colunas que nos interessam
producaoDFQ12 = DataProducaoRaw[['num_docentes', 'num_discentes', 'num_externos', 'paper_id', 'periodico', 'anais_titulo']]
display(producaoDFQ12.info())
producaoDFQ12.head()

In [ ]:
# Existem valores sem docente e externo. Como queremos coautoria com externos, precisamos de pelo menos um participante para
# pelo menos um deles. Logo, retiremos essas tupla.
display(producaoDFQ12.loc[(producaoDFQ12['num_docentes'] == 0) & (producaoDFQ12['num_discentes'] == 0)])

producaoDFQ12 = producaoDFQ12.drop(producaoDFQ12.index[[1961, 2502, 2532, 2540, 2605, 2645, 2666, 2790]])
producaoDFQ12 = producaoDFQ12.reset_index(drop = True)
producaoDFQ12.info()

In [ ]:
# Declarando e inicializando as variáveis que usaremos
artigosExternoPercent = 0
trabalhosExternoPercent = 0
artigosTotal = 0
trabalhosTotal = 0

# Fazendo a contagem do total de artigos e trabalhos e de coautoria de externo para cada um deles
for indx in range(0, len(producaoDFQ12)):
    if producaoDFQ12['periodico'][indx] is not np.nan:
        artigosTotal += 1
        if producaoDFQ12['num_externos'][indx] != 0:
            artigosExternoPercent += 1
    if producaoDFQ12['anais_titulo'][indx] is not np.nan:
        trabalhosTotal += 1
        if producaoDFQ12['num_externos'][indx] != 0:
            trabalhosExternoPercent += 1

# Calculando as porcentagens
artigosExternoPercent = (artigosExternoPercent / artigosTotal) * 100
trabalhosExternoPercent = (trabalhosExternoPercent / trabalhosTotal) * 100

print("\nArtigos de periódico com coautoria de externo: ", artigosExternoPercent, "%\nTrabalhos em anais com coautoria de externo: ", trabalhosExternoPercent, "%")

In [ ]:
# Montando gráficos pie chart para cada resultado
labels = 'Com externo', 'Sem externo'
explode=(0, 0.05)

# Artigos de periódico com coautoria de externo
figure(1, figsize=(6,6))
fracs = [artigosExternoPercent, 100-artigosExternoPercent]
pie(fracs, explode=explode, labels=labels, autopct='%1.4f%%', shadow=True, startangle=90)
title('Taxa de coautoria de externos em artigos de periódico', bbox={'facecolor':'0.8', 'pad':5})

show()

# Trabalhos em anais com coautoria de externo
figure(1, figsize=(6,6))
fracs2 = [trabalhosExternoPercent, 100-trabalhosExternoPercent]
pie(fracs2, explode=explode, labels=labels, autopct='%1.4f%%', shadow=True, startangle=90)
title('Taxas de coautoria de externos em trabalhos em anais', bbox={'facecolor':'0.8', 'pad':5})

show()

## 13) Qual é a distribuição de artigos de periódico publicados em 2017, por estrato do Qualis?

In [ ]:
# Usaremos DataProducaoRaw e DataQualisRaw para responder a pergunta
# Fazendo a limpeza
producaoDFQ13 = DataProducaoRaw[['paper_id', 'periodico', 'issn']]
producaoDFQ13 = producaoDFQ13.dropna(subset = ['periodico', 'issn'])
producaoDFQ13 = producaoDFQ13.reset_index(drop = True)
display(producaoDFQ13.info())
producaoDFQ13.head()

In [ ]:
# Montando o data frame de resultado
distByEstrato = pd.DataFrame(columns=('Estrato', 'Qtd artigos de periódico'))
distByEstrato['Estrato'] = DataQualisRaw['Estrato'].unique()
distByEstrato['Qtd artigos de periódico'] = 0

# Calculando valores por 'estrato'
for indx in range(0, len(producaoDFQ13)):
    for jndx in range(0, len(DataQualisRaw)):
        if producaoDFQ13['issn'][indx] == DataQualisRaw['ISSN'][jndx]:
            distByEstrato.loc[distByEstrato['Estrato'] == DataQualisRaw['Estrato'][jndx], 'Qtd artigos de periódico'] += 1
            break
            
# O resultado obtido
distByEstrato

In [ ]:
# Montando um gráfico de barras verticais dos resultados obtidos
x_labels = distByEstrato['Estrato'].unique()
x = range(len(x_labels))
y = distByEstrato['Qtd artigos de periódico']

plt.figure(figsize=(5, 5))
plt.title('Distribuição Estrato x Qtd Artigos de periódico')
plt.bar(x, y, align='center', width = 0.5, color=['#cc6644', '#4466cc'])
plt.xticks(x, x_labels)

plt.show()

## 14) Considerando os pesos dos artigos de cada estrato do índice restrito ({'A1': 1, 'A2': 0.85, 'B1': 0.70}), qual o índice restrito relativo de cada programa, considerando apenas os artigos em periódicos, e dividido pelo número de docentes permanentes (ordenado do maior para o menor)?

In [ ]:
# Para responder a questão, usaremos DataQualisRaw, DataProducaoRaw e DataDocentesRaw
# Limpando nossos data frames e pegando apenas os dados que nos interessam
qualisDFQ14 = DataQualisRaw[['ISSN', 'Estrato']]
qualisDFQ14.columns.values[0] = "issn"
qualisDFQ14.columns.values[1] = "estrato"
display(qualisDFQ14.info())
display(qualisDFQ14.head())

producaoDFQ14 = DataProducaoRaw.loc[(DataProducaoRaw['periodico'].isnull() == False)
                                   & (DataProducaoRaw['num_docentes'] != 0)][['programa', 'paper_id', 'dict_paper_autores', 'issn']]
producaoDFQ14 = producaoDFQ14.drop_duplicates(subset='paper_id', keep=False)
producaoDFQ14 = producaoDFQ14.reset_index(drop = True)
producaoDFQ14 = producaoDFQ14.drop('paper_id', 1)
display(producaoDFQ14.info())
display(producaoDFQ14.head())

docentesDFQ14 = DataDocentesRaw.loc[DataDocentesRaw['categoria'] == 'PERMANENTE'][['nome']]
display(docentesDFQ14.info())
docentesDFQ14.head()

In [ ]:
# Montando nosso data frame resultado
indiceRestritoRelativoByPrograma = pd.DataFrame(columns=('Programa', 'Índice restrito relativo', 'Total docentes permanentes'))
indiceRestritoRelativoByPrograma['Programa'] = producaoDFQ14['programa'].unique()
indiceRestritoRelativoByPrograma['Índice restrito relativo'] = 0
indiceRestritoRelativoByPrograma['Total docentes permanentes'] = 0

# Criando dicionário para os valores dos estratos que restringem nosso cálculo
estratoDictionary = {'A1': 1, 'A2': 0.85, 'B1': 0.70}

# Fazendo a contagem do índice relativo em relação ao programa e se há docente que é permanente
for indx in range(0, len(producaoDFQ14)):
    dictionaryList = ast.literal_eval(producaoDFQ14['dict_paper_autores'][indx])
    for dictionary in dictionaryList:
        if (dictionary['categoria'] == 'Docente') & (docentesDFQ14[docentesDFQ14['nome'].str.contains(dictionary['nome'])].empty == False) & (qualisDFQ14[qualisDFQ14['issn'].str.contains(producaoDFQ14['issn'][indx])].empty == False):
            indiceRestritoRelativoByPrograma.loc[indiceRestritoRelativoByPrograma['Programa'] == producaoDFQ14['programa'][indx], 'Total docentes permanentes'] += 1
                    
            estrato = qualisDFQ14.loc[qualisDFQ14['issn'] == producaoDFQ14['issn'][indx], "estrato"].reset_index(drop = True)[0]
            estrato = estrato.strip()
            if estrato in estratoDictionary:
                indiceRestritoRelativoByPrograma.loc[indiceRestritoRelativoByPrograma['Programa'] == producaoDFQ14['programa'][indx], 'Índice restrito relativo'] += estratoDictionary[estrato]
            break

# Calculando com o total de docentes permanentes do respectivo programa
indiceRestritoRelativoByPrograma['Resultado'] = indiceRestritoRelativoByPrograma['Índice restrito relativo'] / indiceRestritoRelativoByPrograma['Total docentes permanentes']

# Nosso data frame resultante
indiceRestritoRelativoByPrograma = indiceRestritoRelativoByPrograma.sort_values(['Resultado', 'Índice restrito relativo'], ascending=[False, False])
indiceRestritoRelativoByPrograma = indiceRestritoRelativoByPrograma.reset_index(drop = True)
indiceRestritoRelativoByPrograma

In [ ]:
# Montando resultado em gráfico de barras horizontais com o respectivo valor no final de cada barra
matplotlib.rcParams.update({'font.size': 14})

fig, ax = plt.subplots(figsize=(18,8))
y_pos = [i for i in range(0,len(indiceRestritoRelativoByPrograma['Programa']))]
x_values = list(indiceRestritoRelativoByPrograma['Resultado'])
bar_width = 0.5

ax.set_title("\'Índice restrito relativo/Qtd docentes permanentes\' x Programa")
ax.barh(y_pos, x_values, height=bar_width)
ax.set_yticks(y_pos)
ax.set_yticklabels(list(indiceRestritoRelativoByPrograma['Programa']))
plt.xlim((0,max(x_values)*1.2))
rects = ax.patches

# For each bar: Place a label
for rect in rects:
    # Get X and Y placement of label from rect.
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2

    # Use Y value as label and format number with one decimal place
    label = "{:}".format(x_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(5, 5),              #  Shift label (horizontally,vertically)
        textcoords="offset points", # Interpret `xytext` as offset in points
        ha='left',                  # Horizontal label alignment
        va='top')                   # Vertical label alignment

plt.gca().invert_yaxis()

plt.show()

## 15) Considerando os pesos dos artigos de cada estrato do índice geral ({'A1': 1, 'A2': 0.85, 'B1': 0.70, 'B2': 0.50, 'B3': 0.20, 'B4': 0.10, 'B5': 0.05, 'C': 0.0}), qual o índice geral relativo de cada programa, considerando apenas os artigos em periódicos, e dividido pelo número de docentes permanentes (ordenado do maior para o menor)?

In [ ]:
# Para responder a questão, usaremos DataQualisRaw, DataProducaoRaw e DataDocentesRaw
# Limpando nossos data frames e pegando apenas os dados que nos interessam
qualisDFQ15 = DataQualisRaw[['ISSN', 'Estrato']]
qualisDFQ15.columns.values[0] = "issn"
qualisDFQ15.columns.values[1] = "estrato"
display(qualisDFQ15.info())
display(qualisDFQ15.head())

producaoDFQ15 = DataProducaoRaw.loc[(DataProducaoRaw['periodico'].isnull() == False)
                                   & (DataProducaoRaw['num_docentes'] != 0)][['programa', 'paper_id', 'dict_paper_autores', 'issn']]
producaoDFQ15 = producaoDFQ15.drop_duplicates(subset='paper_id', keep=False)
producaoDFQ15 = producaoDFQ15.reset_index(drop = True)
producaoDFQ15 = producaoDFQ15.drop('paper_id', 1)
display(producaoDFQ15.info())
display(producaoDFQ15.head())

docentesDFQ15 = DataDocentesRaw.loc[DataDocentesRaw['categoria'] == 'PERMANENTE'][['nome']]
display(docentesDFQ15.info())
docentesDFQ15.head()

In [ ]:
# Montando nosso data frame resultado
indiceGeralRelativoByPrograma = pd.DataFrame(columns=('Programa', 'Índice geral relativo', 'Total docentes permanentes'))
indiceGeralRelativoByPrograma['Programa'] = producaoDFQ15['programa'].unique()
indiceGeralRelativoByPrograma['Índice geral relativo'] = 0
indiceGeralRelativoByPrograma['Total docentes permanentes'] = 0

# Criando dicionário para os valores dos estratos
estratoDictionary = {'A1': 1, 'A2': 0.85, 'B1': 0.70, 'B2': 0.50, 'B3': 0.20, 'B4': 0.10, 'B5': 0.05, 'C': 0.0}

# Fazendo a contagem do índice geral em relação ao programa e se há docente que é permanente
for indx in range(0, len(producaoDFQ15)):
    dictionaryList = ast.literal_eval(producaoDFQ15['dict_paper_autores'][indx])
    for dictionary in dictionaryList:
        if (dictionary['categoria'] == 'Docente') & (docentesDFQ15[docentesDFQ15['nome'].str.contains(dictionary['nome'])].empty == False) & (qualisDFQ15[qualisDFQ15['issn'].str.contains(producaoDFQ15['issn'][indx])].empty == False):
            indiceGeralRelativoByPrograma.loc[indiceGeralRelativoByPrograma['Programa'] == producaoDFQ15['programa'][indx], 'Total docentes permanentes'] += 1
                    
            estrato = qualisDFQ15.loc[qualisDFQ15['issn'] == producaoDFQ15['issn'][indx], "estrato"].reset_index(drop = True)[0]
            estrato = estrato.strip()
            indiceGeralRelativoByPrograma.loc[indiceGeralRelativoByPrograma['Programa'] == producaoDFQ15['programa'][indx], 'Índice geral relativo'] += estratoDictionary[estrato]
            break

# Calculando com o total de docentes permanentes do respectivo programa
indiceGeralRelativoByPrograma['Resultado'] = indiceGeralRelativoByPrograma['Índice geral relativo'] / indiceGeralRelativoByPrograma['Total docentes permanentes']

# Nosso data frame resultante
indiceGeralRelativoByPrograma = indiceGeralRelativoByPrograma.sort_values(['Resultado', 'Índice geral relativo'], ascending=[False, False])
indiceGeralRelativoByPrograma = indiceGeralRelativoByPrograma.reset_index(drop = True)
indiceGeralRelativoByPrograma

In [ ]:
# Montando resultado em gráfico de barras horizontais com o respectivo valor no final de cada barra
matplotlib.rcParams.update({'font.size': 14})

fig, ax = plt.subplots(figsize=(18,8))
y_pos = [i for i in range(0,len(indiceGeralRelativoByPrograma['Programa']))]
x_values = list(indiceGeralRelativoByPrograma['Resultado'])
bar_width = 0.5

ax.set_title("\'Índice geral relativo/Qtd docentes permanentes\' x Programa")
ax.barh(y_pos, x_values, height=bar_width)
ax.set_yticks(y_pos)
ax.set_yticklabels(list(indiceGeralRelativoByPrograma['Programa']))
plt.xlim((0,max(x_values)*1.2))
rects = ax.patches

# For each bar: Place a label
for rect in rects:
    # Get X and Y placement of label from rect.
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2

    # Use Y value as label and format number with one decimal place
    label = "{:}".format(x_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(5, 5),              #  Shift label (horizontally,vertically)
        textcoords="offset points", # Interpret `xytext` as offset in points
        ha='left',                  # Horizontal label alignment
        va='top')                   # Vertical label alignment

plt.gca().invert_yaxis()

plt.show()